## Постановка задачи
Загрузим данные, приведем их к числовым, заполним пропуски, нормализуем данные и оптимизируем память.

Разделите выборку на обучающую/проверочную в соотношении 80/20.

Построим параллельный ансамбль (бэггинг) решающих деревьев, используя случайный лес.

Проведем предсказание и проверим качество через каппа-метрику.

Данные:
* https://video.ittensive.com/machine-learning/prudential/train.csv.gz

Соревнование: https://www.kaggle.com/c/prudential-life-insurance-assessment/

© ITtensive, 2020

In [1]:
GRAIN = 11
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import cohen_kappa_score, confusion_matrix, make_scorer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn import preprocessing
from etl_utils import reduce_mem_usage


data = pd.read_csv("https://video.ittensive.com/machine-learning/prudential/train.csv.gz")

data['Product_Info_2_1'] = data['Product_Info_2'].str.slice(0, 1)
data['Product_Info_2_2'] = pd.to_numeric(data['Product_Info_2'].str.slice(1, 2))
data = data.drop('Product_Info_2', axis='columns')

onehot_df = pd.get_dummies(data['Product_Info_2_1'])
onehot_df.columns = ['Product_Info_2_1' + column for column in onehot_df.columns]
data = pd.merge(left=data, right=onehot_df, left_index=True, right_index=True).drop('Product_Info_2_1', axis=1).fillna(-1)
del onehot_df

### Набор столбцов для расчета

In [3]:
columns_groups = ['Insurance_History', 'InsurеdInfo', 'Medical_Keyword', 'Family_Hist', 'Medical_History', 'Product_Info']
columns = ['Wt', 'Ht', 'Ins_Age', 'BMI']
for cg in columns_groups:
    columns.extend(data.columns[data.columns.str.startswith(cg)])
    
columns

['Wt',
 'Ht',
 'Ins_Age',
 'BMI',
 'Insurance_History_1',
 'Insurance_History_2',
 'Insurance_History_3',
 'Insurance_History_4',
 'Insurance_History_5',
 'Insurance_History_7',
 'Insurance_History_8',
 'Insurance_History_9',
 'Medical_Keyword_1',
 'Medical_Keyword_2',
 'Medical_Keyword_3',
 'Medical_Keyword_4',
 'Medical_Keyword_5',
 'Medical_Keyword_6',
 'Medical_Keyword_7',
 'Medical_Keyword_8',
 'Medical_Keyword_9',
 'Medical_Keyword_10',
 'Medical_Keyword_11',
 'Medical_Keyword_12',
 'Medical_Keyword_13',
 'Medical_Keyword_14',
 'Medical_Keyword_15',
 'Medical_Keyword_16',
 'Medical_Keyword_17',
 'Medical_Keyword_18',
 'Medical_Keyword_19',
 'Medical_Keyword_20',
 'Medical_Keyword_21',
 'Medical_Keyword_22',
 'Medical_Keyword_23',
 'Medical_Keyword_24',
 'Medical_Keyword_25',
 'Medical_Keyword_26',
 'Medical_Keyword_27',
 'Medical_Keyword_28',
 'Medical_Keyword_29',
 'Medical_Keyword_30',
 'Medical_Keyword_31',
 'Medical_Keyword_32',
 'Medical_Keyword_33',
 'Medical_Keyword_34',
 

### Нормализация данных

In [4]:
scaler = preprocessing.StandardScaler()
data_transformed = pd.DataFrame(scaler.fit_transform(data[columns]))
columns_transformed = data_transformed.columns
data_transformed['Response'] = data['Response']
data_transformed = reduce_mem_usage(data_transformed)
data_transformed.info()

Потребление памяти меньше на 40.49 Мб (-75.1%)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59381 entries, 0 to 59380
Columns: 119 entries, 0 to Response
dtypes: float16(118), int8(1)
memory usage: 13.4 MB


### Разделение данных
Преобразуем выборки в отдельные наборы данных

In [5]:
data_train, data_test = train_test_split(data_transformed, test_size=.2)
data_train.head()

,0,1,2,3,4,5,6,7,8,9,...,109,110,111,112,113,114,115,116,117,Response
15370,-1.476562,-1.934570,1.349609,-0.833008,0.611816,-0.169434,0.862305,-1.013672,0.861328,-0.928711,...,-0.083679,0.441650,-0.149292,2.134766,1.604492,-0.14209,-0.128906,-1.332031,-0.215942,5
42923,-0.161133,0.514160,-0.997070,-0.471680,0.611816,-0.169434,0.862305,-1.013672,0.874512,-0.928711,...,-0.083679,-2.263672,-0.149292,1.200195,1.604492,-0.14209,-0.128906,-1.332031,-0.215942,8
2813,-0.255127,1.003906,0.592285,-0.842285,-1.634766,-0.169434,0.862305,-1.013672,0.861816,0.100891,...,-0.083679,0.441650,-0.149292,-0.666992,1.604492,-0.14209,-0.128906,-1.332031,-0.215942,8
6481,-0.654297,-0.465576,0.743652,-0.520508,-1.634766,-0.169434,0.862305,-1.013672,0.861816,0.100891,...,-0.083679,0.441650,-0.149292,-1.133789,1.604492,-0.14209,-0.128906,-1.332031,-0.215942,8
16865,-0.278564,0.269287,0.213989,-0.472656,0.611816,-0.169434,0.862305,-1.013672,0.864258,-0.928711,...,-0.083679,-2.263672,-0.149292,-0.200073,-0.623535,-0.14209,-0.128906,0.750977,-0.215942,6


### Перекрестная проверка случайного леса
Каждое дерево (по умолчанию, их 100) строится на своей части выборки со своим набором параметров (max_features). Решение принимается путем голосования деревьев.

Например, 10 деревьев для 1 строки (кортежа) исходных параметров дали следующие классы и их вероятности:

{1:0.5, 1:0.8, 2:0.9, 3:0.7, 5:0.5, 1:0.4, 2:0.5, 6:0.5, 3:0.4, 1:0.95}

По итогам голосования выбирается самый популярный класс, это 1 в данном случае.

Если в случайном лесу слишком много деревьев, то точность предсказания будет меньше, чем у одного, полностью обученного дерева. Число деревьев (estimators) должно соответствовать количеству классов в предсказании (class), размеру выборки (N) и числу разбиений (fold). Примерная формула:
estimators = N / (20-100) / fold / class

В нашем случае, N=60000, fold=5, class=8 => estimators=15...75

In [6]:
x = data_train[columns_transformed]
model = RandomForestClassifier(
    random_state=GRAIN,
    n_estimators=77,
    max_depth=17,
    max_features=27,
    min_samples_leaf=20
)

Диапазон тестирования параметров модели ограничен только вычислительной мощностью. Для проверки модели имеет смысл провести индивидуальные перекрестные проверки для каждого параметра в отдельности, затем в итоговой проверке перепроверить самые лучшие найденные параметры с отклонением +/-10%.

In [7]:
tree_params = {
    'max_depth': range(15, 17),
    'max_features': range(26, 28),
    'n_estimators': range(75, 77),
    'min_samples_leaf': range(19, 21)
}
tree_grid = GridSearchCV(model, tree_params, cv=5, n_jobs=-1, verbose=True, scoring=make_scorer(cohen_kappa_score))
tree_grid.fit(x, data_train['Response'])

Fitting 5 folds for each of 16 candidates, totalling 80 fits


GridSearchCV(cv=5,
             estimator=RandomForestClassifier(max_depth=17, max_features=27,
                                              min_samples_leaf=30,
                                              n_estimators=77,
                                              random_state=11),
             n_jobs=-1,
             param_grid={'max_depth': range(15, 17),
                         'max_features': range(26, 28),
                         'min_samples_leaf': range(19, 21),
                         'n_estimators': range(75, 77)},
             scoring=make_scorer(cohen_kappa_score), verbose=True)

Выведем самые оптимальные параметры и построим итоговую модель

In [8]:
print(tree_grid.best_params_)
model = RandomForestClassifier(
    random_state=GRAIN,
    min_samples_leaf=tree_grid.best_params_['min_samples_leaf'],
    max_features=tree_grid.best_params_['max_features'],
    max_depth=tree_grid.best_params_['max_depth'],
    n_estimators=tree_grid.best_params_['n_estimators']
).fit(x, data_train['Response'])

{'max_depth': 15, 'max_features': 27, 'min_samples_leaf': 19, 'n_estimators': 76}


### Предсказание данных и оценка модели

In [10]:
data_test['target'] = model.predict(data_test[columns_transformed])

Кластеризация дает 0.192, kNN(100) - 0.3, лог. регрессия - 0.512/0.496, SVM - 0.95, реш. дерево - 0.3

In [11]:
print("Случайный лес:", round(cohen_kappa_score(data_test["target"], data_test["Response"], weights="quadratic"), 3))

Случайный лес: 0.487


### Матрица неточностей

In [12]:
print("Случайный лес\n",
      confusion_matrix(data_test["target"], data_test["Response"]))

Случайный лес
 [[ 111   63   15   11   20   34    9    3]
 [ 174  292   10    0   97   73    5    3]
 [  11   11   52    7    1    1    0    0]
 [  52   40   62  187    0    3    1    2]
 [ 121  161   23    0  588   86   10    6]
 [ 319  292   37   24  209 1237  331  137]
 [ 139  122    1    3   62  265  634  115]
 [ 301  316    5   27  115  550  610 3681]]
